In [ ]:
# default_exp data_collection_linkedin_job_list

# data_collection_linkedin_job_list

> API details.

In [ ]:
#hide
# from nbdev.showdoc import *

In [ ]:
import re, time, requests
import pandas as pd
from parsel import Selector
from selenium import webdriver
from selenium.webdriver import *
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By

import config

In [ ]:
def get_driver(url):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(
        executable_path=config.PATH_CHROME_DRIVER, 
        options=chrome_options)
    driver.get(url)
    return driver

def get_ref_id(job_url):
    try:
        found = re.search('(?<=refId=).*(?=&trackingId)', job_url).group()
    except AttributeError:
        found = ''
    return found

def get_track_id(job_url):
    try:
        found = re.search('(?<=trackingId=)(.*)(?=&position)', job_url).group()
    except AttributeError:
        found = ''
    return found

def get_job_info(selector):
    title = selector.css('h3::text').get().strip()
    published = selector.xpath("//time/@datetime").get()
    job_url = selector.xpath("//a/@href").get()    
    idx = get_track_id(job_url)
    return [idx, title, published, job_url]
    
def get_job_list(driver):
    job_list = driver.find_elements_by_xpath("//ul[contains(@class,'jobs-search__results-list')]/li")    
    for job in job_list:
        time.sleep(0.2)
        selector = Selector(text=job.get_attribute("outerHTML"))
        yield get_job_info(selector)        

In [ ]:
for url in config.URLS_LINKEDIN:
    driver = get_driver(url)
    job_list = list(get_job_list(driver))
    jobs = pd.DataFrame(job_list, columns=['id','title','published', 'url'])
    jobs.to_csv(config.FILE_JOBLIST_LK, mode='a', index=False)

In [ ]:
jobs_df = pd.read_csv(config.FILE_JOBLIST_LK)
jobs_df.shape

(116, 4)